# Adversarial Networks Survey - Image Collection

After performing the initial training, the dataset will need to be perturbed based on the adversarial method and the model used. In this case generating a perturbed dataset for each model for each method.

## Setup

In [1]:
import numpy as np
import matplotlib.pyplot as plt

import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler

import copy
import time
import os
from tqdm import tqdm

from adversarialmethods import *

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

transform = transforms.Compose([
    transforms.Resize((70,70)),
    transforms.RandomCrop((64,64)),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])

batch_size = 1

trainset = torchvision.datasets.CIFAR10('./data', download=True, train=True, transform=transform)
trainset, valset = torch.utils.data.random_split(trainset, [40000, 10000])
valload = torch.utils.data.DataLoader(valset, batch_size=batch_size, shuffle=False, num_workers=1)

testset = torchvision.datasets.CIFAR10('./data', download=True, train=False, transform=transform)
testload = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=1)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [3]:
transformUnnormalize = transforms.Compose({
    transforms.Normalize((0, 0, 0), (1/0.229, 1/0.224, 1/0.225)),
    transforms.Normalize((-0.485, -0.456, -0.406), (1, 1, 1))
})

In [4]:
traintupple = (40000, 3, 64, 64)
valtupple = (10000, 3, 64, 64)
testtupple = (10000, 3, 64, 64)

In [5]:
alexnet = torchvision.models.alexnet(weights=torchvision.models.AlexNet_Weights.IMAGENET1K_V1)
num_feats = alexnet.classifier[6].in_features
alexnet.classifier[6] = nn.Linear(num_feats, 10)

googlenet = torchvision.models.googlenet(weights=torchvision.models.GoogLeNet_Weights.IMAGENET1K_V1)
num_feats = googlenet.fc.in_features
googlenet.fc = nn.Linear(num_feats, 10)

mobilenet = torchvision.models.mobilenet_v3_large(weights=torchvision.models.MobileNet_V3_Large_Weights.IMAGENET1K_V1)
num_feat = mobilenet.classifier[3].in_features
mobilenet.classifier[3] = nn.Linear(num_feat, 10)

if not os.path.isfile("models/models.pt"):
    raise FileNotFoundError("File 'models.pt' not found in folder models. Please run initialTraining.ipynb to generate the file needed! (File not included due to size)")

trained_weights_cifar10 = torch.load("models/models.pt")
alexnet.load_state_dict(trained_weights_cifar10["alexnet_state_dict"])
googlenet.load_state_dict(trained_weights_cifar10["googlenet_state_dict"])
mobilenet.load_state_dict(trained_weights_cifar10["mobilenet_state_dict"])

models = [alexnet, googlenet, mobilenet]
modelNames = ["alexnet", "googlenet", "mobilenet"]

In [6]:
def fgsm_collection(dataload, model, tupplesize, device="cpu"):
    perturbedImgCollection = torch.zeros(tupplesize)
    perturbedLabelCollection = torch.zeros(tupplesize[0])
    gtLabelCollection = torch.zeros(tupplesize[0])

    model.eval()
    i = 0
    for _, enumdata in enumerate(tqdm(dataload)):
        data, target = enumdata
        data, target = data.to(device), target.to(device)

        data.requires_grad = True
        output = model(data)
        loss = F.nll_loss(output, target)
        model.zero_grad()
        loss.backward()
        data_grad = data.grad.data
        perturbedImg = fgsm_attack(data, 0.02, data_grad)
        output = model(perturbedImg)
        final_pred = output.max(1, keepdim=True)[1]
        perturbedLabelCollection[i] = final_pred.cpu().item()
        gtLabelCollection[i] = target.cpu().item()
        perturbedImgCollection[i] = perturbedImg.cpu()

        i = i + 1
    
    return perturbedImgCollection, perturbedLabelCollection, gtLabelCollection

def jsma_collection(dataload, model, tupplesize, device="cpu"):
    perturbedImgCollection = torch.zeros(tupplesize)
    perturbedLabelCollection = torch.zeros(tupplesize[0])
    gtLabelCollection = torch.zeros(tupplesize[0])

    model.eval()
    i = 0
    for _, enumdata in enumerate(tqdm(dataload)):
        data, target = enumdata
        data, target = data.to(device), target.to(device)
        data.requires_grad = True

        perturbedImg, final_pred = jsma_attack(model, data, target)
        gtLabelCollection[i] = target.cpu().item()
        perturbedLabelCollection[i] = final_pred
        perturbedImgCollection[i] = perturbedImg.cpu()

        i = i + 1
    
    return perturbedImgCollection, perturbedLabelCollection, gtLabelCollection

def deepfool_collection(dataload, model, tupplesize, device="cpu"):
    perturbedImgCollection = torch.zeros(tupplesize)
    perturbedLabelCollection = torch.zeros(tupplesize[0])
    gtLabelCollection = torch.zeros(tupplesize[0])

    model.eval()
    i = 0
    for _, enumdata in enumerate(tqdm(dataload)):
        data, target = enumdata
        data, target = data.to(device), target.to(device)

        _, _, _, final_pred, perturbedImg = deepfool(data.squeeze(0), model)
        gtLabelCollection[i] = target.cpu().item()
        perturbedLabelCollection[i] = final_pred
        perturbedImgCollection[i] = perturbedImg.cpu()

        i = i + 1
    
    return perturbedImgCollection, perturbedLabelCollection, gtLabelCollection

## Alexnet Collection

In [7]:
alexnet.to(device)

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

### DeepFool

In [8]:
perturbedImgCollection, perturbedLabelCollection, gtLabelCollection = deepfool_collection(valload, alexnet, valtupple, device)

torch.save({
    'perturbedImgCollection': perturbedImgCollection,
    'perturbedLabelCollection': perturbedLabelCollection,
    'gtLabelCollection': gtLabelCollection
}, "perturbedDataset/alexnet_deepfool_train.pt")

perturbedImgCollection, perturbedLabelCollection, gtLabelCollection = deepfool_collection(testload, alexnet, testtupple, device)

torch.save({
    'perturbedImgCollection': perturbedImgCollection,
    'perturbedLabelCollection': perturbedLabelCollection,
    'gtLabelCollection': gtLabelCollection
}, "perturbedDataset/alexnet_deepfool_test.pt")

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10000/10000 [13:56<00:00, 11.96it/s]


### FGSM

In [9]:
perturbedImgCollection, perturbedLabelCollection, gtLabelCollection = fgsm_collection(valload, alexnet, valtupple, device)

torch.save({
    'perturbedImgCollection': perturbedImgCollection,
    'perturbedLabelCollection': perturbedLabelCollection,
    'gtLabelCollection': gtLabelCollection
}, "perturbedDataset/alexnet_fgsm_train.pt")

perturbedImgCollection, perturbedLabelCollection, gtLabelCollection = fgsm_collection(testload, alexnet, testtupple, device)

torch.save({
    'perturbedImgCollection': perturbedImgCollection,
    'perturbedLabelCollection': perturbedLabelCollection,
    'gtLabelCollection': gtLabelCollection
}, "perturbedDataset/alexnet_fgsm_test.pt")

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10000/10000 [00:58<00:00, 170.30it/s]


### JSMA

In [ ]:
perturbedImgCollection, perturbedLabelCollection, gtLabelCollection = jsma_collection(valload, alexnet, valtupple, device)

torch.save({
    'perturbedImgCollection': perturbedImgCollection,
    'perturbedLabelCollection': perturbedLabelCollection,
    'gtLabelCollection': gtLabelCollection
}, "perturbedDataset/alexnet_jsma_train.pt")

 91%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▎              | 9112/10000 [24:06<02:23,  6.19it/s]

In [ ]:
perturbedImgCollection, perturbedLabelCollection, gtLabelCollection = jsma_collection(testload, alexnet, testtupple, device)
torch.save({
    'perturbedImgCollection': perturbedImgCollection,
    'perturbedLabelCollection': perturbedLabelCollection,
    'gtLabelCollection': gtLabelCollection
}, "perturbedDataset/alexnet_jsma_test.pt")

## GoogLeNet Collection

In [8]:
googlenet.to(device)

GoogLeNet(
  (conv1): BasicConv2d(
    (conv): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (conv2): BasicConv2d(
    (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (conv3): BasicConv2d(
    (conv): Conv2d(64, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(192, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (inception3a): Inception(
    (branch1): BasicConv2d(
      (conv): Conv2d(192, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track

### Deepfool

In [9]:
perturbedImgCollection, perturbedLabelCollection, gtLabelCollection = deepfool_collection(valload, googlenet, valtupple, device)

torch.save({
    'perturbedImgCollection': perturbedImgCollection,
    'perturbedLabelCollection': perturbedLabelCollection,
    'gtLabelCollection': gtLabelCollection
}, "perturbedDataset/googlenet_deepfool_train.pt")

perturbedImgCollection, perturbedLabelCollection, gtLabelCollection = deepfool_collection(testload, googlenet, testtupple, device)

torch.save({
    'perturbedImgCollection': perturbedImgCollection,
    'perturbedLabelCollection': perturbedLabelCollection,
    'gtLabelCollection': gtLabelCollection
}, "perturbedDataset/googlenet_deepfool_test.pt")

  9%|██████████████                                                                                                                                                     | 859/10000 [04:48<51:13,  2.97it/s]


KeyboardInterrupt: 

### FGSM

In [ ]:
perturbedImgCollection, perturbedLabelCollection, gtLabelCollection = fgsm_collection(valload, googlenet, valtupple, device)

torch.save({
    'perturbedImgCollection': perturbedImgCollection,
    'perturbedLabelCollection': perturbedLabelCollection,
    'gtLabelCollection': gtLabelCollection
}, "perturbedDataset/googlenet_fgsm_train.pt")

perturbedImgCollection, perturbedLabelCollection, gtLabelCollection = fgsm_collection(testload, googlenet, testtupple, device)

torch.save({
    'perturbedImgCollection': perturbedImgCollection,
    'perturbedLabelCollection': perturbedLabelCollection,
    'gtLabelCollection': gtLabelCollection
}, "perturbedDataset/googlenet_fgsm_test.pt")

### JSMA

In [ ]:
perturbedImgCollection, perturbedLabelCollection, gtLabelCollection = jsma_collection(valload, googlenet, valtupple, device)

torch.save({
    'perturbedImgCollection': perturbedImgCollection,
    'perturbedLabelCollection': perturbedLabelCollection,
    'gtLabelCollection': gtLabelCollection
}, "perturbedDataset/googlenet_jsma_train.pt")

In [ ]:
perturbedImgCollection, perturbedLabelCollection, gtLabelCollection = jsma_collection(testload, googlenet, testtupple, device)

torch.save({
    'perturbedImgCollection': perturbedImgCollection,
    'perturbedLabelCollection': perturbedLabelCollection,
    'gtLabelCollection': gtLabelCollection
}, "perturbedDataset/googlenet_jsma_test.pt")

## MobileNet V3 Large Collection

In [ ]:
mobilenet.to(device)

### DeepFool

In [ ]:
perturbedImgCollection, perturbedLabelCollection, gtLabelCollection = deepfool_collection(valload, mobilenet, valtupple, device)

torch.save({
    'perturbedImgCollection': perturbedImgCollection,
    'perturbedLabelCollection': perturbedLabelCollection,
    'gtLabelCollection': gtLabelCollection
}, "perturbedDataset/mobilenet_deepfool_train.pt")

perturbedImgCollection, perturbedLabelCollection, gtLabelCollection = deepfool_collection(testload, mobilenet, testtupple, device)

torch.save({
    'perturbedImgCollection': perturbedImgCollection,
    'perturbedLabelCollection': perturbedLabelCollection,
    'gtLabelCollection': gtLabelCollection
}, "perturbedDataset/mobilenet_deepfool_test.pt")

### FGSM

In [ ]:
perturbedImgCollection, perturbedLabelCollection, gtLabelCollection = fgsm_collection(valload, mobilenet, valtupple, device)

torch.save({
    'perturbedImgCollection': perturbedImgCollection,
    'perturbedLabelCollection': perturbedLabelCollection,
    'gtLabelCollection': gtLabelCollection
}, "perturbedDataset/mobilenet_fgsm_train.pt")

perturbedImgCollection, perturbedLabelCollection, gtLabelCollection = fgsm_collection(testload, mobilenet, testtupple, device)

torch.save({
    'perturbedImgCollection': perturbedImgCollection,
    'perturbedLabelCollection': perturbedLabelCollection,
    'gtLabelCollection': gtLabelCollection
}, "perturbedDataset/mobilenet_fgsm_test.pt")

### JSMA

In [ ]:
perturbedImgCollection, perturbedLabelCollection, gtLabelCollection = jsma_collection(valload, mobilenet, valtupple, device)

torch.save({
    'perturbedImgCollection': perturbedImgCollection,
    'perturbedLabelCollection': perturbedLabelCollection,
    'gtLabelCollection': gtLabelCollection
}, "perturbedDataset/mobilenet_jsma_train.pt")

In [ ]:
perturbedImgCollection, perturbedLabelCollection, gtLabelCollection = jsma_collection(testload, mobilenet, testtupple, device)

torch.save({
    'perturbedImgCollection': perturbedImgCollection,
    'perturbedLabelCollection': perturbedLabelCollection,
    'gtLabelCollection': gtLabelCollection
}, "perturbedDataset/mobilenet_jsma_test.pt")